In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import geopy.distance
import gc
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

import data_prep_functions
import request_api_nba_stats

In [2]:
base_ini = pd.read_csv("../bases_nba_stats/aux_files/base_nba_per_game_16_17.csv")

In [3]:
all_games_16_17 = data_prep_functions.prepara_base(base_ini)

In [4]:
all_games_16_17 = data_prep_functions.cria_features(all_games_16_17)

In [5]:
geral_16_17 = data_prep_functions.gera_last_N_games(all_games_16_17)

In [6]:
data_prep_functions.get_last_games(all_games_16_17, 
                                   geral_16_17[geral_16_17.TEAM_NAME_home == "Pacers"].tail(1).DATE.iloc[0]
                                   , "IND")[["team_home", "team_away", "PTS_home", "PTS_away", 
                                             "FGA_home", "FGA_away", "fl_home_win"]]

,team_home,team_away,PTS_home,PTS_away,FGA_home,FGA_away,fl_home_win
GAME,,,,,,,
IND @ CLE 2017-04-02,CLE,IND,135,130,95,105,1
TOR @ IND 2017-04-04,IND,TOR,108,90,89,76,1
MIL @ IND 2017-04-06,IND,MIL,104,89,80,71,1
IND @ ORL 2017-04-08,ORL,IND,112,127,84,86,0
IND @ PHI 2017-04-10,PHI,IND,111,120,75,85,0


In [7]:
geral_16_17[geral_16_17.TEAM_NAME_home == "Pacers"].tail(1)["FGA_home_L5"]

ATL @ IND 2017-04-12    89.0
Name: FGA_home_L5, dtype: float64

In [8]:
cross_16_17 = data_prep_functions.variaveis_delta(geral_16_17)

In [9]:
cross_16_17.to_csv("../bases_nba_stats/delta_cross_L5_nba_16_17.csv")

In [11]:
cross_16_17.fl_home_win.mean()

0.583739837398374

In [115]:
datetime.strptime(str(get_date_place("0021800270", year = "2018")[0]), '%Y-%m-%d')

datetime.datetime(2018, 11, 23, 0, 0)

In [8]:
games_ids = get_list_gameids_until() 

In [ ]:
df_full, df_full_jogo, erros = get_nba_stats_data(games_ids)

In [36]:
base_nba_2018 = pd.concat(df_full_jogo).reset_index()

In [37]:
base_nba_2018.to_csv("base_nba_2018_raw.csv")

In [38]:
all_games = data_prep_functions.prepara_base(base_nba_2018)

In [41]:
data_prep_functions.cria_features(all_games)

In [43]:
resp = data_prep_functions.gera_last_N_games(all_games, N=[5])

In [46]:
delta_L5_2018 = data_prep_functions.variaveis_delta(resp)

In [62]:
delta_L5_2018.to_csv("delta_cross_L5_nba_18_19.csv")

### Fluxo Predição no Dia

In [53]:
new_games = get_list_gameids(max_date = datetime.today(), year = '2018', start_at=len(all_games))

In [54]:
df_full, df_full_jogo, erros = get_nba_stats_data(new_games)

2018
Length mismatch: Expected axis has 0 elements, new values have 28 elements
Length mismatch: Expected axis has 0 elements, new values have 28 elements


In [96]:
new_games = pd.concat(df_full_jogo).reset_index()

In [97]:
new_games = data_prep_functions.prepara_base(new_games).reset_index()

In [108]:
new_games

,GAME,GAME_ID_home,TEAM_ID_home,TEAM_NAME_home,team_home,TEAM_CITY_home,FGM_home,FGA_home,FG_PCT_home,FG3M_home,...,DATE,SEASON,fl_playoff,fl_home_win,team_home_game_num,team_away_game_num,DISTANCE_KM_home,DISTANCE_KM_away,DAYS_FROM_LAST_GAME_home,DAYS_FROM_LAST_GAME_away
0,WAS @ GSW 2018-10-24,0021800062,1610612744,Warriors,GSW,Golden State,53,92,0.576,15,...,2018-10-24,2019,0,1,1,1,0.000000,3912.001793,2,2
1,CLE @ DET 2018-10-25,0021800063,1610612765,Pistons,DET,Detroit,None,None,None,None,...,2018-10-25,2019,0,0,1,1,0.000000,185.392825,2,1
2,POR @ ORL 2018-10-25,0021800064,1610612753,Magic,ORL,Orlando,None,None,None,None,...,2018-10-25,2019,0,0,1,1,1794.410551,4069.325464,3,3


In [107]:
new_games = data_prep_functions.cria_features(new_games, all_games)

In [110]:
to_predict = data_prep_functions.gera_last_N_games(new_games, all_games)

In [111]:
to_predict = data_prep_functions.variaveis_delta(to_predict)

In [112]:
to_predict[['D2_PFD_L5', 'D2_NET_RATING_L5', 'D2_EFG_PCT_L5', 'C1_TO_L5']]

,D2_PFD_L5,D2_NET_RATING_L5,D2_EFG_PCT_L5,C1_TO_L5
WAS @ GSW 2018-10-24,4.0,5.26,0.1168,4.8
CLE @ DET 2018-10-25,-0.6,4.88,0.0034,-4.8
POR @ ORL 2018-10-25,9.0,-5.36,0.1734,1.2
